# Testing out some features on the Twitter API using Tweepy

In [1]:
import json
import tweepy
import datetime as dt
import pandas as pd

In [2]:
with open('keys.json') as keys:
    config = json.load(keys)
auth = tweepy.OAuthHandler(config['twitter']['API_key'], config['twitter']['API_secret_key'])
# auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

## Example account (Elon Musk)

In [3]:
user = api.get_user('elonmusk')

In [4]:
print(user.screen_name)
print(user.followers_count)
for friend in user.friends():
    print(friend.screen_name)

elonmusk
49493389
TheMarsSociety
CoinDesk
dogecoin_devs
dogecoin
cleantechnica
TheOnion
TheBabylonBee
karpathy
Astro_Soichi
PopMech
PyTorch
Nigel_Lockyer
jagarikin
AstroVicGlover
Tesmanian_com
flcnhvy
machineIearning
BBC_Future
kanyewest
teslacn


## Search by keyword on all tweets

In [4]:
# tweets = tweepy.Cursor(user.search, q=str(search_string), tweet_mode='extended', lang='pt').items(ct.num_of_tweets)
# tweet_search = api.search()
query = '$GME'
max_tweets = 100000
searched_tweets = [status for status in tweepy.Cursor(api.search, q=query, since="2021-2-22", until="2021-2-28").items()]

In [14]:
print(searched_tweets[0])

Status(_api=<tweepy.api.API object at 0x000001D8C8A44288>, _json={'created_at': 'Wed Mar 24 00:54:02 +0000 2021', 'id': 1374524877093314569, 'id_str': '1374524877093314569', 'text': 'RT @FortuneMaster4: $JAKK reiterating my final PT of $30 here, definitely has room to go way higher. Fundamentals are solid with massive ch…', 'truncated': False, 'entities': {'hashtags': [], 'symbols': [{'text': 'JAKK', 'indices': [20, 25]}], 'user_mentions': [{'screen_name': 'FortuneMaster4', 'name': 'Fortune Master', 'id': 1286142595715018757, 'id_str': '1286142595715018757', 'indices': [3, 18]}], 'urls': []}, 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'}, 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'in_reply_to_status_id': None, 'in_reply_to_status_id_str': None, 'in_reply_to_user_id': None, 'in_reply_to_user_id_str': None, 'in_reply_to_screen_name': None, 'user': {'id': 770830615, 'id_str': '770830615', 'name': 'John Goldman', 'scr

Looks like there is a lot of information to dig into here. We might be able to just search on 'symbols' since this seems to already have the stock symbols taged. For now we'll just get a rough count of the number of tweets in the past week.

In [6]:
# since="2021-2-22", max_tweets = 100000
print(len(searched_tweets))

1044


In [8]:
# since="2021-2-22"
print(len(searched_tweets))

1045


In [10]:
# since="2021-2-22", until="2021-3-23"
print(len(searched_tweets))

1045


In [15]:
# since="2021-2-22", until=None
print(len(searched_tweets))

1070


So we had 917 tweets in the past week with the $GNUS tag. Now we need to get this in a format we can use with pandas.

In [ ]:
stock = 'GNUS'
start_date = dt.date(year=2021, month=2, day=22)
end_date = dt.date(year=2021, month=2, day=28)

searched_tweets = [status for status in tweepy.Cursor(api.search, q="${}".format(stock), since=start_date, until=end_date).items(max_tweets)]
print(len(searched_tweets))

In [16]:
def get_stock_tweets(stock, start_date, end_date=None, max_tweets=100000):
    searched_tweets = [status for status in tweepy.Cursor(api.search, q="${}".format(stock), since=start_date, until=end_date).items(max_tweets)]
    return searched_tweets

In [18]:
start_date = dt.datetime(year=2021, month=2, day=22)
end_date = dt.datetime(year=2021, month=3, day=22)
tweets = get_stock_tweets('GNUS', start_date)
print(len(tweets))

12


In [ ]:
# Test of function above
start_date = dt.datetime(year=2021, month=2, day=25)
end_date = dt.datetime(year=2021, month=2, day=28)
get_stock_tweets('GNUS', start_date, end_date)

In [ ]:
starting_stocks = ['GNUS', 'XSPA', 'IBIO', 'GME', 'OPES']

In [ ]:
start_date = dt.datetime(year=2021, month=3, day=15)
end_date = dt.datetime(year=2021, month=3, day=22)
data = pd.DataFrame()
for stock in starting_stocks:
    tweets = get_stock_tweets(stock, start_date, end_date, max_tweets=)
    
    stock_data = pd.DataFrame([{'datetime':tweet.created_at, 'tweet':tweet._json['text'], 'stock': stock} for tweet in tweets])
    data = data.append(stock_data)
data = data.set_index('datetime')

In [ ]:
data.head()

In [ ]:
tweets_bucketed = data.groupby([pd.Grouper(freq='5min'), 'stock']).count().reset_index()
tweets_bucketed.head()

In [ ]:
tweets_bucketed[tweets_bucketed['stock'] == 'XSPA']

In [ ]:
tweets_bucketed.to_parquet('tweet_count.parquet')

## Generate tweets df using multiple API calls

In [ ]:
starting_stocks = ['GNUS', 'XSPA', 'IBIO', 'GME', 'OPES']

In [ ]:
def generate_intervals(minutes_interval, start_time, end_time):
    total_minutes = (end_time - start_time).total_seconds()/60
    return [end_time - dt.timedelta(minutes=x) for x in range(0, int(total_minutes), minutes_interval)]

In [ ]:
start_date = dt.datetime(year=2021, month=2, day=22)
end_date = dt.datetime(year=2021, month=2, day=28)
generate_intervals(5, start_date, end_date)

In [ ]:
start_date = dt.datetime(year=2021, month=2, day=22)
end_date = dt.datetime(year=2021, month=2, day=28)

for stock in starting_stocks:
    print(stock)
    for time in generate_intervals(5, start_date, end_date):
        print(time.isoformat())
        stock_tweet_num = len(get_stock_tweets(stock, time, time + dt.timedelta(minutes=5)))
        print(stock_tweet_num)

## Testing functions for twitter_data.py

In [27]:
def get_stock_tweets_list(stock, start_date, end_date=None, max_tweets=None):
    searched_tweets = [status for status in tweepy.Cursor(api.search, q="${}".format(stock), since=start_date, until=end_date).items()]
    return searched_tweets

In [32]:
def generate_df(stocks, start_date, end_date=None):
    data = pd.DataFrame()
    for stock in stocks:
        tweets = get_stock_tweets_list(stock, start_date, end_date)
        json_data = [tweet._json for tweet in tweets]
        stock_data = pd.io.json.json_normalize(json_data)
        stock_data['stock'] = stock
        data = data.append(stock_data)
    data = data.set_index('created_at')
    return data

In [35]:
start_date = dt.datetime(year=2021, month=2, day=22)
end_date = dt.datetime(year=2021, month=2, day=28)
test_df = generate_df(['GNUS'], start_date)

In [36]:
print(test_df.shape)

(12, 286)
